# Hugging Face Causal Language Model

fine-tuning of the [Facebook/opt-125](https://huggingface.co/facebook/opt-125m) model using an portuguese dataset [mc4-pt-sample-1g.txt](https://unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt) of 300 million tokens in its causal language modeling pre-training. The opt-125 model was originally trained on a english dataset of approximately 300 bil\lion tokens.

The task is to predict the next token in a sequence, like GPT and not like BERT

[![google colab link](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tcvieira/IA368-DD-012023/blob/main/assingments/04/notebook.ipynb)

In [2]:
# The training is done using an T4 with 16GB of memory
!nvidia-smi

Wed Mar 29 19:30:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Installs

In [3]:
!pip install transformers -q
!pip install datasets -q
!pip install ipython-autotime -q
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 112.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.4 MB/s eta 0:00:00
time: 379 µs (started: 2023-03-29 

# Imports

In [4]:
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    AutoConfig
)
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

time: 6.41 s (started: 2023-03-29 19:30:37 +00:00)


# Dataset

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive
time: 30.3 s (started: 2023-03-29 19:30:43 +00:00)


In [6]:
PATH_DATASET = '/content/drive/MyDrive/unicamp/IA368DD/class_4'

time: 342 µs (started: 2023-03-29 19:31:14 +00:00)


In [7]:
#!gsutil cp gs://unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt {PATH_DATASET}/sample-1gb.txt

time: 245 µs (started: 2023-03-29 19:31:14 +00:00)


In [8]:
!head {PATH_DATASET}/sample-1gb.txt

Linkbar Há alguns anos, o número de rapazes e moças que subiam ao púlpito para pregar era maior que o de hoje. Na sua simplicidade, falavam do amor de Deus, da Salvação e davam testemunho sob a unção do Espirito Santo. Hoje, parece que a figura do "preletor oficial" inibiu muitos de falarem com ousadia a Palavra de Deus. Parece que há um receio de falar diante de um público que, certamente, é mais intelectualizado que há alguns anos. Jovens pregadores ficam embaraçados e cometem certos deslizes, que poderiam ser evitados. Neste modesto trabalho, vamos dar apenas algumas sugestões, e não um estudo sobre a Homilética (Arte de Falar em Publico). I -O QUE PREGAR? É a comunicação verbal da Palavra de Deus aos ouvintes. É a transmissão do evangelho de Nosso Senhor Jesus Cristo às pessoas que precisam ouvi-lo. II- QUAL A FINALIDADE DA PREGAÇÃO? É persuadir as pessoas a aceitarem a mensagem da Palavra de Deus para sua salvação (descrentes) ou para seu crescimento espiritual (crentes). Diante d

In [9]:
!wc -l {PATH_DATASET}/sample-1gb.txt

250000 /content/drive/MyDrive/unicamp/IA368DD/class_4/sample-1gb.txt
time: 12.8 s (started: 2023-03-29 19:31:16 +00:00)


## small dataset for testing

In [10]:
#!sed -n '1,100p' {PATH_DATASET}/sample-1gb.txt > {PATH_DATASET}/sample_small.txt

time: 211 µs (started: 2023-03-29 19:31:29 +00:00)


In [11]:
!wc -l {PATH_DATASET}/sample_small.txt

100 /content/drive/MyDrive/unicamp/IA368DD/class_4/sample_small.txt
time: 1.03 s (started: 2023-03-29 19:31:29 +00:00)


In [12]:
small_dataset = load_dataset("text", data_files=f'{PATH_DATASET}/sample_small.txt')
small_dataset["validation"] = load_dataset("text", data_files=f'{PATH_DATASET}/sample_small.txt', split=f"train[:5%]")
small_dataset["train"] = load_dataset("text", data_files=f'{PATH_DATASET}/sample_small.txt', split=f"train[5%:]")

small_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-9e0a1430f318dd3d/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 95
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 5
    })
})

time: 3.18 s (started: 2023-03-29 19:31:30 +00:00)


In [ ]:
# base_dataset = load_dataset("text", data_files=f'{PATH_DATASET}/sample-1gb.txt')
# base_dataset["validation"] = load_dataset("text", data_files=f'{PATH_DATASET}/sample-1gb.txt', split=f"train[:20%]")
# base_dataset["train"] = load_dataset("text", data_files=f'{PATH_DATASET}/sample-1gb.txt', split=f"train[20%:]")

# Select Dataset

In [13]:
dataset = small_dataset

time: 564 µs (started: 2023-03-29 19:41:40 +00:00)


# Parameters

In [14]:
MODEL_NAME = 'facebook/opt-125m'
MAX_SEQ_LENGTH=256
BATCH_SIZE=16
EPOCHS=1
MODEL_OUTPUT_FOLDER=f'{PATH_DATASET}/model_output'
MODEL_SAVE_FOLDER=f'{PATH_DATASET}/model_save'
CONTEXT_LENGTH=256
BATCH_SIZE=4

time: 814 µs (started: 2023-03-29 19:44:48 +00:00)


# Tokenizer

In [15]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

time: 13.2 s (started: 2023-03-29 19:44:53 +00:00)


# Model

`AutoModelForCausalLM` is used for auto-regressive language models like all the GPT models, while `AutoModelForSeq2SeqLM` is used for language models with encoder-decoder architecture like T5 and BART.

https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForCausalLM

In [16]:
# Download configuration from huggingface.co and cache.
config = AutoConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_config(config)
model.to(device)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

time: 7.23 s (started: 2023-03-29 19:45:08 +00:00)


In [17]:
model_size = sum(t.numel() for t in model.parameters())
print(f"OPT-125m size: {model_size/1000**2:.1f}M parameters")

OPT-125m size: 125.2M parameters
time: 1.58 ms (started: 2023-03-29 19:45:25 +00:00)


# Tokenization

The model was pretrained with context length of 2048. We use length 256 initially to overcome the exceeded ram problem.

CHANGE: we tokenize all samples in the batch (consisting of 1000 documents) and create a long sequence of tokens by concatenating all examples and separating them with the special EOS token. Finally, we divide the long sequence into chunks of 512 tokens, which will be used for training.

In [18]:
tokenized_dataset = dataset.map(lambda x: tokenizer(x["text"], 
                                      truncation=True, 
                                      padding="max_length", 
                                      max_length=CONTEXT_LENGTH), 
                                      batched=True, 
                                      num_proc=4, 
                                      remove_columns=["text"]
                                     )

Map (num_proc=4):   0%|          | 0/95 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/5 [00:00<?, ? examples/s]

time: 876 ms (started: 2023-03-29 19:45:51 +00:00)


In [19]:
print(tokenized_dataset)
print(f"{len(tokenized_dataset['train']['input_ids'][0])} tokens - {tokenized_dataset['train']['input_ids'][0]}")

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 95
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 5
    })
})
256 tokens - [2, 2571, 29766, 3849, 6382, 13572, 673, 230, 13974, 47047, 121, 1916, 4065, 11387, 263, 10, 10770, 17682, 293, 3304, 1526, 2953, 2664, 6417, 15357, 242, 117, 5766, 22415, 2102, 263, 9194, 281, 263, 21691, 5224, 366, 263, 11605, 4, 381, 1634, 3286, 26464, 1021, 21033, 271, 11988, 8101, 207, 263, 8446, 1342, 366, 295, 4214, 14258, 1210, 6510, 23473, 34867, 808, 4216, 2664, 3175, 853, 281, 385, 281, 6641, 281, 263, 11605, 4, 7879, 162, 1020, 109, 998, 6, 9, 2816, 1755, 119, 9, 2399, 281, 25948, 20559, 364, 1198, 3381, 366, 10, 19348, 1977, 548, 354, 263, 21691, 5224, 366, 364, 23358, 263, 1076, 8913, 366, 4, 2193, 221, 1526, 571, 1243, 109, 13663, 1794, 221, 1526, 571, 1243, 3137, 4252, 366, 326, 1140, 438, 12979, 366, 109, 14242, 1794, 3137, 139, 27705, 106

# Training

In [20]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir=MODEL_OUTPUT_FOLDER,
                                  num_train_epochs=EPOCHS, 
                                  per_device_train_batch_size=BATCH_SIZE,
                                  per_device_eval_batch_size=BATCH_SIZE, 
                                  evaluation_strategy="epoch", 
                                  save_strategy="epoch",
                                  logging_strategy="epoch", 
                                  learning_rate=2e-5, 
                                  weight_decay=0.01,
                                  fp16=True
                                )

time: 3.47 ms (started: 2023-03-29 19:46:06 +00:00)


In [21]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(model=model, 
                  args=training_args, 
                  train_dataset=tokenized_dataset["train"],
                  eval_dataset=tokenized_dataset["validation"], 
                  data_collator=data_collator)

time: 23.1 ms (started: 2023-03-29 19:46:13 +00:00)


In [22]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,10.162800,9.854373


TrainOutput(global_step=24, training_loss=10.162772496541342, metrics={'train_runtime': 11.7302, 'train_samples_per_second': 8.099, 'train_steps_per_second': 2.046, 'total_flos': 12411371520000.0, 'train_loss': 10.162772496541342, 'epoch': 1.0})

time: 11.8 s (started: 2023-03-29 19:46:21 +00:00)


In [23]:
model.save_pretrained(MODEL_SAVE_FOLDER)
tokenizer.save_pretrained(MODEL_SAVE_FOLDER)

('/content/drive/MyDrive/unicamp/IA368DD/class_4/model_output/tokenizer_config.json',
 '/content/drive/MyDrive/unicamp/IA368DD/class_4/model_output/special_tokens_map.json',
 '/content/drive/MyDrive/unicamp/IA368DD/class_4/model_output/vocab.json',
 '/content/drive/MyDrive/unicamp/IA368DD/class_4/model_output/merges.txt',
 '/content/drive/MyDrive/unicamp/IA368DD/class_4/model_output/added_tokens.json',
 '/content/drive/MyDrive/unicamp/IA368DD/class_4/model_output/tokenizer.json')

time: 1.45 s (started: 2023-03-29 19:46:46 +00:00)


In [24]:
!zip model {MODEL_SAVE_FOLDER}/*

  adding: content/drive/MyDrive/unicamp/IA368DD/class_4/model_save/config.json (deflated 49%)
  adding: content/drive/MyDrive/unicamp/IA368DD/class_4/model_save/generation_config.json (deflated 30%)
  adding: content/drive/MyDrive/unicamp/IA368DD/class_4/model_save/pytorch_model.bin (deflated 7%)
time: 27 s (started: 2023-03-29 19:47:35 +00:00)


# Evaluation

In [34]:
model = AutoModelForCausalLM.from_pretrained(MODEL_SAVE_FOLDER)
trainer = Trainer(
    model = model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator = data_collator
)

time: 1.99 s (started: 2023-03-29 19:52:50 +00:00)


In [35]:
eval_results = trainer.evaluate()
perplexity = torch.exp(torch.tensor(eval_results["eval_loss"]))
print(f'Train Perplexity =  {perplexity.item():.2f}')

Train Perplexity =  19041.44
time: 74.8 ms (started: 2023-03-29 19:52:56 +00:00)


In [36]:
eval_results = trainer.evaluate(tokenized_dataset["validation"])
perplexity = torch.exp(torch.tensor(eval_results["eval_loss"]))
print(f'Test Perplexity =  {perplexity.item():.2f}')

Test Perplexity =  19041.44
time: 71.4 ms (started: 2023-03-29 19:52:59 +00:00)


In [39]:
prompt = 'rapazes e moças que subiam ao púlpito'
inputs = tokenizer(prompt, return_tensors='pt')

# Generate
generate_ids = model.generate(inputs.input_ids.to(device), max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'rapazes e moças que subiam ao púlpito de de de de de de de de de de de de de de'

time: 1.87 s (started: 2023-03-29 19:55:02 +00:00)


In [38]:
prompt = 'rapazes e moças que subiam ao púlpito'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenizer(text=prompt)['input_ids']
    input_ids_truncated = input_ids[-CONTEXT_LENGTH:]  # Usamos apenas os últimos tokens como entrada para o modelo.
    output = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = output['logits'][:, -1, :]  # Usamos apenas o ultimo token da sequencia
    predicted_id = torch.argmax(logits).item()  # extraindo o token de maior probabilidade (greedy decoding)
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt.replace('</s>', ''))

rapazes e moças que subiam ao púlpito de
rapazes e moças que subiam ao púlpito de de
rapazes e moças que subiam ao púlpito de de de
rapazes e moças que subiam ao púlpito de de de de
rapazes e moças que subiam ao púlpito de de de de de
rapazes e moças que subiam ao púlpito de de de de de de
rapazes e moças que subiam ao púlpito de de de de de de de
rapazes e moças que subiam ao púlpito de de de de de de de de
rapazes e moças que subiam ao púlpito de de de de de de de de de
rapazes e moças que subiam ao púlpito de de de de de de de de de de
rapazes e moças que subiam ao púlpito de de de de de de de de de de de
rapazes e moças que subiam ao púlpito de de de de de de de de de de de de
rapazes e moças que subiam ao púlpito de de de de de de de de de de de de de
rapazes e moças que subiam ao púlpito de de de de de de de de de de de de de de
rapazes e moças que subiam ao púlpito de de de de de de de de de de de de de de de
rapazes e moças que subiam ao púlpito de de de de de de de de de de de

----